<a href="https://colab.research.google.com/github/nglglhtr/tapiz/blob/master/Reliabilityv2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [5]:
!pip install -q kaggle

In [6]:
from google.colab import files
files.upload()

Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"angelagilhotra","key":"ed43085b3e82faaef4325f0240dfbaec"}'}

In [7]:
! mkdir ~/.kaggle 
! cp kaggle.json ~/.kaggle/
! chmod 600 ~/.kaggle/kaggle.json

In [ ]:
 ! kaggle datasets list

In [9]:
! kaggle competitions download -c fake-news fake-news

 53% 5.00M/9.42M [00:00<00:00, 37.3MB/s]
100% 9.42M/9.42M [00:00<00:00, 60.2MB/s]
  0% 0.00/40.6k [00:00<?, ?B/s]
100% 40.6k/40.6k [00:00<00:00, 35.1MB/s]
 67% 25.0M/37.0M [00:00<00:00, 32.1MB/s]
100% 37.0M/37.0M [00:00<00:00, 54.3MB/s]


In [12]:
! mkdir fake-news
! mv submit.csv test.csv.zip train.csv.zip fake-news


mkdir: cannot create directory ‘fake-news’: File exists
mv: cannot stat 'submit.csv': No such file or directory
mv: cannot stat 'test.csv.zip': No such file or directory
mv: cannot stat 'train.csv.zip': No such file or directory
mkdir: cannot create directory ‘fake-news’: File exists
mv: cannot stat 'submit.csv': No such file or directory
mv: cannot stat 'test.csv.zip': No such file or directory
mv: cannot stat 'train.csv.zip': No such file or directory


In [13]:
! unzip fake-news/test.csv.zip -d fake-news
! unzip fake-news/train.csv.zip -d fake-news

Archive:  fake-news/test.csv.zip
  inflating: fake-news/test.csv      
Archive:  fake-news/train.csv.zip
  inflating: fake-news/train.csv     


In [1]:
import pandas as pd 

train = pd.read_csv('./fake-news/train.csv')
test = pd.read_csv('./fake-news/test.csv')
submit = pd.read_csv('./fake-news/submit.csv')

train = train.set_index('id', drop=True)
test = test.set_index('id', drop=True)

label: 1 => unreliable

In [2]:
train.head()

,title,author,text,label
id,,,,
0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1


In [3]:
test.head()

,title,author,text
id,,,
20800,"Specter of Trump Loosens Tongues, if Not Purse...",David Streitfeld,"PALO ALTO, Calif. — After years of scorning..."
20801,Russian warships ready to strike terrorists ne...,NaN,Russian warships ready to strike terrorists ne...
20802,#NoDAPL: Native American Leaders Vow to Stay A...,Common Dreams,Videos #NoDAPL: Native American Leaders Vow to...
20803,"Tim Tebow Will Attempt Another Comeback, This ...",Daniel Victor,"If at first you don’t succeed, try a different..."
20804,Keiser Report: Meme Wars (E995),Truth Broadcast Network,42 mins ago 1 Views 0 Comments 0 Likes 'For th...


In [4]:
submit.head()

,id,label
0,20800,0
1,20801,1
2,20802,0
3,20803,1
4,20804,1


## Data Preparation

In [91]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.model_selection import train_test_split

from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Embedding, GRU, LSTM, RNN, SpatialDropout1D, Bidirectional

%tensorflow_version 2.x
import tensorflow as tf
device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
  raise SystemError('GPU device not found')
print('Found GPU at: {}'.format(device_name))


Found GPU at: /device:GPU:0


In [6]:
train[['title', 'author']] = train[['title', 'author']].fillna(value='missing')
train=train.dropna()
train.isnull().sum()

title     0
author    0
text      0
label     0
dtype: int64

In [7]:
test[['title', 'author']] = test[['title', 'author']].fillna(value='missing')
test=test.dropna()
test.isnull().sum()

title     0
author    0
text      0
dtype: int64

Removing outliers

In [8]:
length = []
[length.append(len(str(text))) for text in train['text']]
train['length'] = length
train.head()

,title,author,text,label,length
id,,,,,
0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1,4930
1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0,4160
2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1,7692
3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1,3237
4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1,938


In [9]:
train = train.drop(train['text'][train['length'] < 50].index, axis = 0)

In [10]:
train.shape

(20554, 5)

In [11]:
length = []
[length.append(len(str(text))) for text in test['text']]
test['length'] = length
test.head()

,title,author,text,length
id,,,,
20800,"Specter of Trump Loosens Tongues, if Not Purse...",David Streitfeld,"PALO ALTO, Calif. — After years of scorning...",8015
20801,Russian warships ready to strike terrorists ne...,missing,Russian warships ready to strike terrorists ne...,1559
20802,#NoDAPL: Native American Leaders Vow to Stay A...,Common Dreams,Videos #NoDAPL: Native American Leaders Vow to...,4547
20803,"Tim Tebow Will Attempt Another Comeback, This ...",Daniel Victor,"If at first you don’t succeed, try a different...",3578
20804,Keiser Report: Meme Wars (E995),Truth Broadcast Network,42 mins ago 1 Views 0 Comments 0 Likes 'For th...,542


In [14]:
min(train['length']), max(train['length']), round(sum(train['length'])/len(train['length']))

(50, 142961, 4598)

In [15]:
min(test['length']), max(test['length']), round(sum(test['length'])/len(test['length']))

(50, 107014, 4707)

In [63]:
max_features = 4500

In [99]:
tokenizer = Tokenizer(num_words = max_features, filters='!"#$%&()*+,-./:;<=>?@[\\]^_`{|}~\t\n', lower = True, split = ' ', oov_token='<OOV>')
tokenizer.fit_on_texts(texts = train['text'])
X = tokenizer.texts_to_sequences(texts = train['text'])
X = pad_sequences(sequences = X, maxlen = max_features, padding = 'post')
X

array([[ 130,    1, 2307, ...,    0,    0,    0],
       [ 360,  115,    2, ...,    0,    0,    0],
       [ 211,    2,  738, ...,    0,    0,    0],
       ...,
       [   2,    1,    4, ...,    0,    0,    0],
       [1067,  183,    3, ...,    0,    0,    0],
       [ 743,    1,    9, ...,    0,    0,    0]], dtype=int32)

In [77]:
print(X.shape)
y = train['label'].values
print(y.shape)

(20554, 4500)
(20554,)


In [78]:
X_train, X_test, y_train, y_test = train_test_split(
    X, 
    y, 
    test_size = 0.2, random_state = 101)

In [79]:
y_train

array([1, 1, 0, ..., 0, 1, 0])

In [80]:
X_train

array([[1381, 3119, 2675, ...,    0,    0,    0],
       [ 673,   10,  368, ...,    0,    0,    0],
       [ 889,   62, 1162, ...,    0,    0,    0],
       ...,
       [1766,  442,    2, ...,    0,    0,    0],
       [  39,    2, 1084, ...,    0,    0,    0],
       [1682,  138,   42, ...,    0,    0,    0]], dtype=int32)

In [81]:
X_test

array([[ 115,   74,    8, ...,    0,    0,    0],
       [2278,   18,   14, ...,    0,    0,    0],
       [ 755,    3,  340, ...,    0,    0,    0],
       ...,
       [2414, 3463,   35, ...,    0,    0,    0],
       [  39, 1938,  421, ...,    0,    0,    0],
       [ 900,   15, 3346, ...,    0,    0,    0]], dtype=int32)

In [82]:
y_test

array([1, 1, 0, ..., 0, 1, 1])

## building Sequential model

In [94]:

model = Sequential([
                    Embedding(input_dim=max_features, output_dim=120),
                    Bidirectional(LSTM(units=120, 
                         activation='tanh',
                         recurrent_activation='sigmoid',
                         unroll=False,
                         use_bias=True,
                         dropout = 0.2, 
                         recurrent_dropout = 0
                         )),
                    # Dropout(rate = 0.5),
                    Dense(units = 120,  activation = 'relu'),
                    # Dropout(rate = 0.5),
                    Dense(units = len(set(y)),  activation = 'sigmoid', name = 'output_layer')
])


model.summary()


Model: "sequential_13"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_14 (Embedding)     (None, None, 120)         540000    
_________________________________________________________________
bidirectional_2 (Bidirection (None, 240)               231360    
_________________________________________________________________
dense_13 (Dense)             (None, 120)               28920     
_________________________________________________________________
output_layer (Dense)         (None, 2)                 242       
Total params: 800,522
Trainable params: 800,522
Non-trainable params: 0
_________________________________________________________________


In [95]:
model.compile(optimizer = 'adam', loss = 'sparse_categorical_crossentropy', metrics = ['accuracy'])

In [96]:
with tf.device('/device:GPU:0'):
  model_fit = model.fit(
      X_train, y_train, epochs = 1, verbose=1)

514/514 [==============================] - 197s 384ms/step - loss: 0.1998 - accuracy: 0.9127


In [97]:
from sklearn.metrics import accuracy_score
prediction = model.predict_classes(
    X_test
)
accuracy_score(y_test, prediction)

0.9632692775480418

In [114]:
test = pd.read_csv('./fake-news/test.csv')
test = test.set_index('id', drop=True)
test = test.fillna(' ')

In [115]:
test.shape

(5200, 3)

In [116]:
test_sequences = tokenizer.texts_to_sequences(test['text'])
test_sequences = pad_sequences(sequences = test_sequences, maxlen = max_features, padding = 'post')
test_sequences

array([[   1,    1, 3464, ...,    0,    0,    0],
       [ 230,    1, 1290, ...,    0,    0,    0],
       [1561,    1, 1663, ...,    0,    0,    0],
       ...,
       [ 192,  562,  164, ...,    0,    0,    0],
       [   1, 1042,  221, ...,    0,    0,    0],
       [ 664, 2075,  383, ...,    0,    0,    0]], dtype=int32)

In [125]:
type(test['text'])

pandas.core.series.Series

In [117]:
test_prediction = model.predict_classes(
    test_sequences
)
# accuracy_score(y_test, prediction)

In [118]:
submission = pd.DataFrame({'id':test.index, 'label':test_prediction})
submission.shape

(5200, 2)

In [120]:
submission.head()

,id,label
0,20800,0
1,20801,1
2,20802,0
3,20803,0
4,20804,1


In [121]:
submission.to_csv('submission.csv', index=False)

## Scratch space

In [130]:
content = 'U.S. Secretary of State John F. Kerry said Monday that he will stop in Paris later this week, amid criticism that no top American officials attended Sunday’s unity march against terrorism.\n\nKerry said he expects to arrive in Paris Thursday evening, as he heads home after a week abroad. He said he will fly to France at the conclusion of a series of meetings scheduled for Thursday in Sofia, Bulgaria. He plans to meet the next day with Foreign Minister Laurent Fabius and President Francois Hollande, then return to Washington.\n\nThe visit by Kerry, who has family and childhood ties to the country and speaks fluent French, could address some of the criticism that the United States snubbed France in its darkest hour in many years.\n\nThe French press on Monday was filled with questions about why neither President Obama nor Kerry attended Sunday’s march, as about 40 leaders of other nations did. Obama was said to have stayed away because his own security needs can be taxing on a country, and Kerry had prior commitments.\n\nAmong roughly 40 leaders who did attend was Israeli Prime Minister Benjamin Netanyahu, no stranger to intense security, who marched beside Hollande through the city streets. The highest ranking U.S. officials attending the march were Jane Hartley, the ambassador to France, and Victoria Nuland, the assistant secretary of state for European affairs. Attorney General Eric H. Holder Jr. was in Paris for meetings with law enforcement officials but did not participate in the march.\n\nKerry spent Sunday at a business summit hosted by India’s prime minister, Narendra Modi. The United States is eager for India to relax stringent laws that function as barriers to foreign investment and hopes Modi’s government will act to open the huge Indian market for more American businesses.\n\nIn a news conference, Kerry brushed aside criticism that the United States had not sent a more senior official to Paris as “quibbling a little bit.” He noted that many staffers of the American Embassy in Paris attended the march, including the ambassador. He said he had wanted to be present at the march himself but could not because of his prior commitments in India.\n\n“But that is why I am going there on the way home, to make it crystal clear how passionately we feel about the events that have taken place there,” he said.\n\n“And I don’t think the people of France have any doubts about America’s understanding of what happened, of our personal sense of loss and our deep commitment to the people of France in this moment of trauma.'
content2 = """
Just hours after classes resumed at an Indiana school, administrators were notified that a student had tested positive for Covid-19.
The school district - one of the first in the US to reopen - told families that it has now enacted its Covid-19 protocol: the student, who attended classes for part of the day, has been isolated and all close contacts have been identified.
And two states south, in Georgia, the state's largest school district has confirmed that roughly 260 employees have either tested positive for the virus, or have been exposed to infections.
The Gwinnett County School District (GCPS) is set to reopen for online classes on 12 August. But despite the confirmed community spread, parents held protests last week demanding students be allowed to return to school.
The reports come as US schools grapple with how to safely reopen as the school year approaches.
Despite climbing infections across the country, President Donald Trump has urged schools to reopen by fall. 
"""

seq = tokenizer.texts_to_sequences([content2])
padded = pad_sequences(seq, maxlen=max_features, padding='post')
pred = model.predict_classes(padded)
pred

## 0 -> reliable
## 1 -> unreliable

array([0])

## Saving model

In [123]:
model.summary()

Model: "sequential_13"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_14 (Embedding)     (None, None, 120)         540000    
_________________________________________________________________
bidirectional_2 (Bidirection (None, 240)               231360    
_________________________________________________________________
dense_13 (Dense)             (None, 120)               28920     
_________________________________________________________________
output_layer (Dense)         (None, 2)                 242       
Total params: 800,522
Trainable params: 800,522
Non-trainable params: 0
_________________________________________________________________


In [132]:
!mkdir -p saved_model
model.save('reliability_classifier.h5') 

In [131]:
import pickle

# saving
with open('tokenizer.pk', 'wb') as handle:
    pickle.dump(tokenizer, handle, protocol=pickle.HIGHEST_PROTOCOL)